<h2>Capstone Project - Data Analysis on Potential Relocation Site for a Professional Baseball Team in Virgina</h2>

In [1]:
#import libraries
import numpy as np  # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
import json         # library to handle JSON files
import requests     # library to handle requests
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Store the demographic link in a variable. Using the function get in the request library, download the page to a variable

In [3]:
virginia_cities_link = 'https://www.virginia-demographics.com/cities_by_population'
raw_cities_page = requests.get(virginia_cities_link)

Use data attribute text to extract XML to a string & assign it to a page variable

In [4]:
cities_page = raw_cities_page.text

In [5]:
# Extract the table
table_start_point = cities_page.find('<table class="ranklist span8">')
table_end_point = cities_page.find("</table>") + 8  # 8 is the offset for the ending table tag
city_table = cities_page[table_start_point:table_end_point]

In [6]:
city_df = pd.read_html(city_table, header = 0)
city_df = city_df[0]

# Drop the Population Rank column
city_df.drop('Virginia Cities by Population Rank', axis=1, inplace=True)

# Drop the last row in the table
city_df.drop(558, axis = 0, inplace=True)

# Convert Population data type from object to int
city_df['Population'] = city_df['Population'].astype('int64')

# Per league requirements, drop any rows where the population is less than 4000
city_df = city_df.drop(city_df[city_df['Population'] < 4000].index)

# Print the last five rows to check that the drop worked correctly
city_df.tail()

,City,Population
205,Spotsylvania Courthouse,4220
206,Mount Hermon,4209
207,University Center,4199
208,Enon,4063
209,Norton,4043


The existing baseball teams in Virginia were downloaded into a csv file for convenience. Read in the csv file to a second data set 

In [7]:
teams_df = pd.read_csv('teams.csv', header=None)

# Drop columns 1,2,3,5 and 6
teams_df.drop([1,2,3,5,6], axis=1, inplace=True)

# Rename the columns
teams_df.rename(columns={0:'Team Name', 4:'City'}, inplace=True)
teams_df.head()

,Team Name,City
0,Bluefield Blue Jays,Bluefield
1,Bristol Pirates,Bristol
2,Danville Braves,Danville
3,Lynchburg Hillcats,Lynchburg
4,Norfolk Tides,Norfolk


Create a new data set that contains only the cities that do not have an already existing team

In [8]:
# Function to check if the city in teams_df is not in city_df
target_cities_df = city_df[~city_df.City.isin(teams_df.City)]

# Reset the index
target_cities_df.reset_index(drop=True, inplace=True)
target_cities_df.head()

,City,Population
0,Virginia Beach,450057
1,Chesapeake,235410
2,Arlington,229534
3,Newport News,180775
4,Alexandria,154710


<h3>Next, explore neighborhood and clustering for the top 3 cities returned.</h3>

<p><strong>City 1: Virginia Beach</strong></p>

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
# Get the coordinates for Virginia Beach
vb_address = 'Virginia Beach, VA'
geolocator = Nominatim(user_agent = "va_explorer")
location = geolocator.geocode(vb_address)
vb_lat = location.latitude
vb_long = location.longitude
print ('The geographical coordinates of Virginia Beach are {} {}.'.format(vb_lat,vb_long))

The geographical coordinates of Virginia Beach are 36.8529841 -75.9774183.


Generate a map for Virginia Beach

In [11]:
map_virginiabeach = folium.Map(location = [vb_lat, vb_long], zoom_start = 15)

map_virginiabeach

There appears to be a field near Kilbourne Bl. Re-draw the map to get the coordinates of Kilbourne Bl.

In [12]:
vb_address = 'Kilbourne Bl, Virginia Beach, VA'
geolocator = Nominatim(user_agent = "vb_explorer")
location = geolocator.geocode(vb_address)
vb_lat = location.latitude
vb_long = location.longitude
print ('The geographical coordinates of Kilbourne, Bl, Virginia Beach are {} {}.'.format(vb_lat,vb_long))

The geographical coordinates of Kilbourne, Bl, Virginia Beach are 36.8563033 -75.9840296.


Use FourSquare API to search for venues nearby

In [13]:
radius = 500
LIMIT  = 15
searchQuery = 'Restaurant'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, vb_lat, vb_long, VERSION, searchQuery, radius, LIMIT)

In [14]:
results = requests.get(url).json()

In [15]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",False,513a6650e4b01394afc5bef4,500 Pinewood Road,US,Virginia Beach,United States,Holly Road,347,"[500 Pinewood Road (Holly Road), Virginia Beac...","[{'label': 'display', 'lat': 36.85916143354793...",36.859161,-75.982471,23451,VA,Tempte Restaurant & Lounge,v-1551217149,NaN
1,"[{'id': '4bf58dd8d48988d157941735', 'name': 'N...",False,4ba049b6f964a520816637e3,3102 Holly Rd,US,Virginia Beach,United States,32nd St,332,"[3102 Holly Rd (32nd St), Virginia Beach, VA 2...","[{'label': 'display', 'lat': 36.85916876421602...",36.859169,-75.982966,23451,VA,Terrapin Restaurant,v-1551217149,48256649
2,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,51fc39b0498e7a8fef30e895,500 Pinewood Rd.,US,Virginia Beach,United States,NaN,342,"[500 Pinewood Rd., Virginia Beach, VA 23451, U...","[{'label': 'display', 'lat': 36.859105, 'lng':...",36.859105,-75.982438,23451,VA,Tempt Restaurant & Lounge,v-1551217149,NaN
3,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,4ba4ee28f964a5207ec438e3,508 Laskin Rd,US,Virginia Beach,United States,NaN,308,"[508 Laskin Rd, Virginia Beach, VA 23451, Unit...","[{'label': 'display', 'lat': 36.85872734774832...",36.858727,-75.982342,23451,VA,Nick's,v-1551217149,NaN
4,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",False,53835e36498e17831b287803,328 Laskin Rd,US,Virginia Beach,United States,NaN,460,"[328 Laskin Rd, Virginia Beach, VA 23451, Unit...","[{'label': 'display', 'lat': 36.85901877458885...",36.859019,-75.980126,23451,VA,Home Republic Brewpub,v-1551217149,90675779


In [16]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Tempte Restaurant & Lounge,American Restaurant,500 Pinewood Road,US,Virginia Beach,United States,Holly Road,347,"[500 Pinewood Road (Holly Road), Virginia Beac...","[{'label': 'display', 'lat': 36.85916143354793...",36.859161,-75.982471,23451,VA,513a6650e4b01394afc5bef4
1,Terrapin Restaurant,New American Restaurant,3102 Holly Rd,US,Virginia Beach,United States,32nd St,332,"[3102 Holly Rd (32nd St), Virginia Beach, VA 2...","[{'label': 'display', 'lat': 36.85916876421602...",36.859169,-75.982966,23451,VA,4ba049b6f964a520816637e3
2,Tempt Restaurant & Lounge,Bar,500 Pinewood Rd.,US,Virginia Beach,United States,NaN,342,"[500 Pinewood Rd., Virginia Beach, VA 23451, U...","[{'label': 'display', 'lat': 36.859105, 'lng':...",36.859105,-75.982438,23451,VA,51fc39b0498e7a8fef30e895
3,Nick's,Seafood Restaurant,508 Laskin Rd,US,Virginia Beach,United States,NaN,308,"[508 Laskin Rd, Virginia Beach, VA 23451, Unit...","[{'label': 'display', 'lat': 36.85872734774832...",36.858727,-75.982342,23451,VA,4ba4ee28f964a5207ec438e3
4,Home Republic Brewpub,Brewery,328 Laskin Rd,US,Virginia Beach,United States,NaN,460,"[328 Laskin Rd, Virginia Beach, VA 23451, Unit...","[{'label': 'display', 'lat': 36.85901877458885...",36.859019,-75.980126,23451,VA,53835e36498e17831b287803


In [17]:
venues_map = folium.Map(location=[vb_lat, vb_long], zoom_start=15) # generate map centred around the Kilbourne Bl

# add a red circle marker to represent the Kilbourne Bl
folium.features.CircleMarker(
    [vb_lat, vb_long],
    radius=10,
    color='red',
    popup='Kilbourne Bl',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the venues as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<p><strong>City 2: Chesapeake</strong></p>

In [18]:
# Get the coordinates for Chesapeake
address = 'Chesapeake, VA'
geolocator = Nominatim(user_agent = "va_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print ('The geographical coordinates of Chesapeake are {} {}.'.format(latitude,longitude))

The geographical coordinates of Chesapeake are 36.7183708 -76.2466798.


Generate a map of Chesapeake

In [19]:
map_chesapeake = folium.Map(location = [latitude, longitude], zoom_start = 15)

map_chesapeake

No noticeable park, but there does appear to be an empty spot on Albemarle Drive. Use this as the center of the map and get the nearby venues

In [20]:
address = 'Albemarle Drive, Chesapeake, VA'
geolocator = Nominatim(user_agent = "va_explorer")
location = geolocator.geocode(address)
ches_lat = location.latitude
ches_long = location.longitude
print ('The geographical coordinates of Albemarle, Drive, Chesapeake are {} {}.'.format(ches_lat,ches_long))

The geographical coordinates of Albemarle, Drive, Chesapeake are 36.7176183 -76.2493086.


Use Foursquare API to search nearby venues

In [21]:
radius = 500
LIMIT  = 15
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, ches_lat, ches_long, VERSION, radius, LIMIT)

In [22]:
ches_results = requests.get(url).json()


In [23]:
ches_venues = ches_results['response']['venues']
ches_df = json_normalize(ches_venues)
ches_df.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d12e941735', 'name': 'P...",False,4be314da21d5a59300241811,NaN,US,Chesapeake,United States,NaN,53,"[Chesapeake, VA, United States]","[{'label': 'display', 'lat': 36.71799983988239...",36.718000,-76.248942,NaN,VA,Chesapeake Police and Fire/EMS,v-1551217191
1,"[{'id': '4bf58dd8d48988d12b941735', 'name': 'C...",False,4ba0d0abf964a520887f37e3,Albemarle Dr,US,Chesapeake,United States,Holt Dr,210,"[Albemarle Dr (Holt Dr), Chesapeake, VA 23322,...","[{'label': 'display', 'lat': 36.71677236361184...",36.716772,-76.251422,23322,VA,Chesapeake Circuit Court,v-1551217191
2,"[{'id': '4bf58dd8d48988d12b941735', 'name': 'C...",False,4bf4195894af2d7f352d3a72,Albemarle Drive,US,Chesapeake,United States,NaN,155,"[Albemarle Drive, Chesapeake, VA 23322, United...","[{'label': 'display', 'lat': 36.71895238383206...",36.718952,-76.249839,23322,VA,Chesapeake General District Court,v-1551217191
3,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",False,4b63640cf964a52075762ae3,306 Cedar Rd,US,Chesapeake,United States,NaN,169,"[306 Cedar Rd, Chesapeake, VA 23322, United St...","[{'label': 'display', 'lat': 36.71609773928122...",36.716098,-76.249352,23322,VA,Chesapeake City Hall,v-1551217191
4,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",False,4d262d173c84b1f736700947,304 Albemarle Dr,US,Chesapeake,United States,NaN,55,"[304 Albemarle Dr, Chesapeake, VA 23322, Unite...","[{'label': 'display', 'lat': 36.71802166264145...",36.718022,-76.248942,23322,VA,Chesapeake Public Safety Building,v-1551217191


In [24]:
filtered_columns = ['name', 'categories'] + [col for col in ches_df.columns if col.startswith('location.')] + ['id']
ches_df_filtered = ches_df.loc[:, filtered_columns]

# filter the category for each row
ches_df_filtered['categories'] = ches_df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
ches_df_filtered.columns = [column.split('.')[-1] for column in ches_df_filtered.columns]

ches_df_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Chesapeake Police and Fire/EMS,Police Station,NaN,US,Chesapeake,United States,NaN,53,"[Chesapeake, VA, United States]","[{'label': 'display', 'lat': 36.71799983988239...",36.718000,-76.248942,NaN,VA,4be314da21d5a59300241811
1,Chesapeake Circuit Court,Courthouse,Albemarle Dr,US,Chesapeake,United States,Holt Dr,210,"[Albemarle Dr (Holt Dr), Chesapeake, VA 23322,...","[{'label': 'display', 'lat': 36.71677236361184...",36.716772,-76.251422,23322,VA,4ba0d0abf964a520887f37e3
2,Chesapeake General District Court,Courthouse,Albemarle Drive,US,Chesapeake,United States,NaN,155,"[Albemarle Drive, Chesapeake, VA 23322, United...","[{'label': 'display', 'lat': 36.71895238383206...",36.718952,-76.249839,23322,VA,4bf4195894af2d7f352d3a72
3,Chesapeake City Hall,City Hall,306 Cedar Rd,US,Chesapeake,United States,NaN,169,"[306 Cedar Rd, Chesapeake, VA 23322, United St...","[{'label': 'display', 'lat': 36.71609773928122...",36.716098,-76.249352,23322,VA,4b63640cf964a52075762ae3
4,Chesapeake Public Safety Building,Government Building,304 Albemarle Dr,US,Chesapeake,United States,NaN,55,"[304 Albemarle Dr, Chesapeake, VA 23322, Unite...","[{'label': 'display', 'lat': 36.71802166264145...",36.718022,-76.248942,23322,VA,4d262d173c84b1f736700947
5,Chesapeake Magistrate Office,Building,400 Albemarle Dr,US,Chesapeake,United States,NaN,40,"[400 Albemarle Dr, Chesapeake, VA 23322, Unite...","[{'label': 'display', 'lat': 36.71780734597564...",36.717807,-76.249703,23322,VA,4e6b729be4cdb37552022391
6,Chesapeake JDR Court,Courthouse,Albemarle Dr,US,Great Bridges,United States,NaN,130,"[Albemarle Dr, Great Bridges, VA 23322, United...","[{'label': 'display', 'lat': 36.71704979942552...",36.717050,-76.248028,23322,VA,4bd8315fe914a593c99a52fa
7,Chesapeake Jail,Prison,400 Albemarle Dr,US,Chesapeake,United States,NaN,161,"[400 Albemarle Dr, Chesapeake, VA 23322, Unite...","[{'label': 'display', 'lat': 36.71898755987486...",36.718988,-76.249909,23322,VA,4bab89edf964a520a1b13ae3
8,Great Bridge Community Center,Community Center,212 Holt Dr,US,Chesapeake,United States,Shea Dr,225,"[212 Holt Dr (Shea Dr), Chesapeake, VA 23322, ...","[{'label': 'display', 'lat': 36.71652713198722...",36.716527,-76.251436,23322,VA,4c66d641e75ac9288ba7f8da
9,Chesapeake Planetarium,Planetarium,304 Albemarle Dr,US,Chesapeake,United States,Chesapeake Municipal Center,244,[304 Albemarle Dr (Chesapeake Municipal Center...,"[{'label': 'display', 'lat': 36.71588920854981...",36.715889,-76.251004,23322,VA,4cf6e49c69aaa090623a832c


In [25]:
ches_venues_map = folium.Map(location=[ches_lat, ches_long], zoom_start=15) # generate map centred around Albemarle Drive

# add a red circle marker to represent Albemarle Drive
folium.features.CircleMarker(
    [ches_lat, ches_long],
    radius=10,
    color='red',
    popup='Albemarle Drive',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the venues as blue circle markers
for lat, lng, label in zip(ches_df_filtered.lat, ches_df_filtered.lng, ches_df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(ches_venues_map)

# display map
ches_venues_map

<p><strong>City 3: Arlington</strong></p>

In [26]:
# Get the coordinates for Virginia Beach
address = 'Arlington, VA'
geolocator = Nominatim(user_agent = "arg_explorer")
location = geolocator.geocode(address)
arg_lat = location.latitude
arg_long = location.longitude
print ('The geographical coordinates of Arlington are {} {}.'.format(arg_lat,arg_long))

The geographical coordinates of Arlington are 38.8903961 -77.0841585.


In [27]:
map_arlington = folium.Map(location = [arg_lat, arg_long], zoom_start = 15)

map_arlington

Arlington does not appear to have a noticable location for a ballpark. Use FourSquare to check the nearby venues

In [28]:
radius = 500
LIMIT  = 15
searchQuery = 'Restaurant'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, arg_lat, arg_long, VERSION, searchQuery, radius, LIMIT)

In [29]:
arg_results = requests.get(url).json()


In [30]:
arg_venues = arg_results['response']['venues']
arg_df = json_normalize(arg_venues)
arg_df.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d11d941735', 'name': 'S...",False,40b13b00f964a52006f51ee3,1520 N Courthouse Rd,US,Arlington,United States,at Clarendon Blvd.,106,"[1520 N Courthouse Rd (at Clarendon Blvd.), Ar...","[{'label': 'display', 'lat': 38.89134832800888...",38.891348,-77.084018,NaN,22201,VA,Summers Grill Restaurant & Sports Bar,v-1551217262,122305586
1,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,4baf0f5cf964a5203de93be3,2250 Clarendon Blvd Ste G,US,Arlington,United States,NaN,216,"[2250 Clarendon Blvd Ste G, Arlington, VA 2220...","[{'label': 'display', 'lat': 38.889428, 'lng':...",38.889428,-77.086324,Clarendon - Courthouse,22201,VA,Asahi Japanese Restaurant,v-1551217262,NaN
2,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",False,4f3203ef19833175d609e85c,2700 Clarendon Blvd,US,Arlington,United States,NaN,599,"[2700 Clarendon Blvd, Arlington, VA 22201, Uni...","[{'label': 'display', 'lat': 38.888699, 'lng':...",38.888699,-77.090730,NaN,22201,VA,Essential Restaurant Holdings,v-1551217262,NaN
3,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",False,4ad53055f964a5204c0221e3,2250 Clarendon Blvd,US,Arlington,United States,NaN,157,"[2250 Clarendon Blvd, Arlington, VA 22201, Uni...","[{'label': 'display', 'lat': 38.889652, 'lng':...",38.889652,-77.085706,NaN,22201,VA,Sawatdee,v-1551217262,NaN


In [31]:
filtered_columns = ['name', 'categories'] + [col for col in arg_df.columns if col.startswith('location.')] + ['id']
arg_df_filtered = arg_df.loc[:, filtered_columns]

# filter the category for each row
arg_df_filtered['categories'] = arg_df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
arg_df_filtered.columns = [column.split('.')[-1] for column in arg_df_filtered.columns]

arg_df_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Summers Grill Restaurant & Sports Bar,Sports Bar,1520 N Courthouse Rd,US,Arlington,United States,at Clarendon Blvd.,106,"[1520 N Courthouse Rd (at Clarendon Blvd.), Ar...","[{'label': 'display', 'lat': 38.89134832800888...",38.891348,-77.084018,NaN,22201,VA,40b13b00f964a52006f51ee3
1,Asahi Japanese Restaurant,Sushi Restaurant,2250 Clarendon Blvd Ste G,US,Arlington,United States,NaN,216,"[2250 Clarendon Blvd Ste G, Arlington, VA 2220...","[{'label': 'display', 'lat': 38.889428, 'lng':...",38.889428,-77.086324,Clarendon - Courthouse,22201,VA,4baf0f5cf964a5203de93be3
2,Essential Restaurant Holdings,Food,2700 Clarendon Blvd,US,Arlington,United States,NaN,599,"[2700 Clarendon Blvd, Arlington, VA 22201, Uni...","[{'label': 'display', 'lat': 38.888699, 'lng':...",38.888699,-77.090730,NaN,22201,VA,4f3203ef19833175d609e85c
3,Sawatdee,Thai Restaurant,2250 Clarendon Blvd,US,Arlington,United States,NaN,157,"[2250 Clarendon Blvd, Arlington, VA 22201, Uni...","[{'label': 'display', 'lat': 38.889652, 'lng':...",38.889652,-77.085706,NaN,22201,VA,4ad53055f964a5204c0221e3


In [32]:
arg_venues_map = folium.Map(location=[arg_lat, arg_long], zoom_start=15) # generate map centred around Albemarle Drive

# add a red circle marker to represent Albemarle Drive
folium.features.CircleMarker(
    [arg_lat, arg_long],
    radius=10,
    color='red',
    popup='Arlington',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(arg_venues_map)

# add the venues as blue circle markers
for lat, lng, label in zip(arg_df_filtered.lat, arg_df_filtered.lng, arg_df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(arg_venues_map)

# display map
arg_venues_map